In [41]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
import torch
import torch.nn as nn
import matplotlib.pyplot as plt

ImportError: cannot import name 'FillMissing' from 'fastai.tabular' (C:\Users\adrie\AppData\Local\Programs\Python\Python39\lib\site-packages\fastai\tabular\__init__.py)

In [32]:
df = pd.read_csv("CarvanaClean.csv")

In [33]:
df.head()

,Unnamed: 0,model,details,mileage,Bluetooth,Backup Camera,Infotainment,Screen,Navigation,Hands Free Calling,...,4-Cyl,3-Cyl,V6,V8,6-Cyl,city_mpg,highway_mpg,price,year,make
0,0,Altima,2.5 SR,57543,1,1,1,0,0,1,...,1,0,0,0,0,26,37,19990,2016,Nissan
1,1,Civic,LX,51210,0,0,1,0,0,0,...,1,0,0,0,0,25,36,16990,2011,Honda
2,2,Focus,Titanium,19216,1,1,1,0,0,0,...,1,0,0,0,0,26,38,21990,2016,Ford
3,3,Passat,1.8T Wolfsburg Edition,54167,1,0,1,0,0,1,...,1,0,0,0,0,24,34,17590,2014,Volkswagen
4,4,EcoSport,S,11168,1,1,1,0,0,0,...,1,0,0,0,0,23,29,23990,2021,Ford


In [34]:
df.drop('Unnamed: 0', axis=1, inplace=True)

In [35]:
len(df)

7294

In [19]:
# X = df_dummies.drop('price', axis=1).to_numpy()
# y = df_dummies['price'].to_numpy()
# X = StandardScaler().fit_transform(X)
# pca = PCA(n_components=10)

# pca_X = pca.fit_transform(X)
# pca_df = pd.DataFrame(data= pca_X)
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [36]:
df.columns

Index(['model', 'details', 'mileage', 'Bluetooth', 'Backup Camera',
       'Infotainment', 'Screen', 'Navigation', 'Hands Free Calling',
       'Heated Seats', 'Power Seat Controls', 'Rear Air Vents', 'Bed Liner',
       'Tow Hitch', '4-Cyl', '3-Cyl', 'V6', 'V8', '6-Cyl', 'city_mpg',
       'highway_mpg', 'price', 'year', 'make'],
      dtype='object')

In [37]:
cat_feateures = ['model', 'details', 'Bluetooth', 'Backup Camera',
       'Infotainment', 'Screen', 'Navigation', 'Hands Free Calling',
       'Heated Seats', 'Power Seat Controls', 'Rear Air Vents', 'Bed Liner',
       'Tow Hitch', '4-Cyl', '3-Cyl', 'V6', 'V8','make']

In [38]:
cont_features = ['mileage', 'city_mpg', 'highway_mpg', 'year']

In [39]:
procs=[FillMissing, Categorify, Normalize]

NameError: name 'FillMissing' is not defined